In [1]:
## VISSIM Modules
import win32com.client as com
import os

## RL Modules
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("ERROR: GPU DEVICE NOT FOUND.")

from keras.models import load_model
    
## Data Management Modules
import pickle

## User Defined Modules
import Simulator_Functions as SF
from RLAgents import DQNAgent
from NParser import NetworkParser
from COMServer import COMServerDispatch, COMServerReload
from TupleToList import toList
from Utilities import log_progress, pltlive
## Other Modules
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

Default GPU Device: /device:GPU:0


Using TensorFlow backend.


In [2]:
## RL Hyperparamenters
# Number of simulations, save every "n" episodes and copy weights with frequency "f"
episodes = 500
partial_save_at = 100
copy_weights_frequency = 5

# Timesteps per simulation (1 timestep = 0.1 sec), length for random population is a multiple of episode
timesteps_per_second = 1
seconds_per_green = 6
seconds_per_yellow = 3
simulation_length = 3600*1 + 1

## State-Action Parameters
action_type = "phases"        # options are "phases" and "programs"
state_size = 4
action_size = 8

# Hyperparameters
PER_activated = True
batch_size = 256
memory_size = 1000
alpha   = 0.001
gamma   = 0.95
memory_population_length = 10*memory_size

# Exploration Schedule ("linear" or "geometric")
exploration_schedule = "geometric"
epsilon_start = 1
epsilon_end   = 0.001
if exploration_schedule == "linear":
    epsilon_decay = 1.2*(epsilon_end - epsilon_start)/(episodes-1)
    epsilon_sequence = [1 + epsilon_decay * entry for entry in range(episodes+1)]
    epsilon_sequence = [0 if entry < 0 else entry for entry in epsilon_sequence]
elif exploration_schedule == "geometric":
    epsilon_decay = np.power(epsilon_end/epsilon_start, 1./(episodes-1)) # Geometric decay
    epsilon_sequence = [epsilon_start * epsilon_decay ** entry for entry in range(episodes+1)]
else:
    print("ERROR: Unrecognized choice of exploration schedule.")

# Demand Schedule (times in seconds, demand in cars/hour as PPP)
demand_change_timesteps = 450
demand = {"h":800, 'm':400, 'l':200}
demand_list = [[demand['l'], demand['l']], [demand['m'], demand['l']],\
              [demand['h'], demand['l']], [demand['h'], demand['m']],\
              [demand['h'], demand['h']], [demand['m'], demand['h']],
              [demand['l'], demand['h']], [demand['l'], demand['m']]]

In [3]:
## Operation mode (selects functionalities)
mode = "debug"
# "populate" = population of memory, generation of initial memory file
# "training" = training agents, maximum speed, frozen UI, mid amount of messages
# "debug"    = trains for 1 episode, minimum speed, working UI, all messages
# "demo"     = loads pretrained agent, minimum speed, working UI
# "test"     = executes evaluation, maximum speed

## Network Model Parameters

model_name  = 'Balance'
# 'Single_Cross_Straight'
# 'Single_Cross_Triple'
# 'Balance'

vissim_working_directory = 'C:\\Users\\acabrejasegea\\OneDrive - The Alan Turing Institute\\Desktop\\ATI\\0_TMF\\MLforFlowOptimisation\\Vissim\\'
agent_type = 'DuelingDDQN'         # DQN, DuelingDQN, DDQN, DuelingDDQN
reward_type = 'Queues'   
# 'Queues'          Sum of the queues for all lanes in intersection
# 'QueuesDiff'      Difference in queue lengths in last timestep
# 'QueuesDiffSC'    10000* QueuesDiff - Queues^2
# 'TotalDelayDiff'
state_type  = 'Queues'    
Random_Seed = 42

## Use of additional files?
flag_read_additionally  = False
SaveResultsAgent = True
# Random demand
Random_Demand = False

# Session ID
Session_ID = 'Ep_'+str(episodes)+'_A_'+agent_type+"_Act_"+action_type+"_Rew_"+reward_type

In [4]:
# Initialize simulation
Vissim, Simulation, Network, cache_flag = COMServerDispatch(model_name, vissim_working_directory,\
                                                            memory_population_length, timesteps_per_second,\
                                                            delete_results = True, verbose = True)


Working Directory set to: C:\Users\acabrejasegea\OneDrive - The Alan Turing Institute\Desktop\ATI\0_TMF\MLforFlowOptimisation\Vissim\
Generating Cache...
Cache generated.

****************************
*   COM Server dispatched  *
****************************

Attempting to load Model File: Balance.inpx ...
Load process successful
Simulation length set to 10000 seconds.
Results from Previous Simulations: Deleted. Fresh Start Available.
Fetched and containerized Simulation Object
Fetched and containerized Network Object 

*******************************************************
*                                                     *
*                 SETUP COMPLETE                      *
*                                                     *
*******************************************************



In [5]:
npa = NetworkParser(Vissim)

In [6]:
npa.signal_lanes

[['10028-1',
  '10025-2',
  '10026-1',
  '10020-1',
  '10030-2',
  '10031-1',
  '103-1',
  '105-1',
  '107-1',
  '109-1',
  '111-1',
  '101-1'],
 ['10037-1',
  '10040-2',
  '10041-1',
  '10033-1',
  '10043-2',
  '10042-1',
  '113-1',
  '115-1'],
 ['10054-1',
  '10044-2',
  '10048-1',
  '10055-3',
  '10050-1',
  '10047-2',
  '10046-1',
  '10052-3',
  '117-1',
  '119-1',
  '121-1',
  '123-1',
  '125-1',
  '127-1',
  '129-1',
  '131-1'],
 ['10061-1',
  '10064-2',
  '10059-1',
  '10057-1',
  '10067-2',
  '10066-1',
  '135-1',
  '137-1',
  '139-1',
  '141-1',
  '143-1',
  '133-1'],
 ['10069-1', '10071-2', '10073-2', '10072-1', '145-1'],
 ['10078-1', '10075-1', '10074-1', '147-1', '149-1', '151-1'],
 ['10086-1', '10084-2', '10080-1', '10087-2', '153-1', '155-1'],
 ['10093-1',
  '10091-1',
  '10096-2',
  '10090-1',
  '10094-2',
  '157-1',
  '159-1',
  '161-1',
  '163-1',
  '165-1',
  '167-1'],
 ['10100-1', '10102-1', '10101-1', '169-1', '171-1', '173-1', '175-1'],
 ['10109-1',
  '10113-1',
  

In [7]:
Agents = [DQNAgent(state_size, action_size, ID, state_type, npa, memory_size,\
                           gamma, epsilon_sequence[0], alpha, copy_weights_frequency, Vissim, PER_activated,\
                           DoubleDQN = True if agent_type == "DDQN" or "DuelingDDQN" else False,\
                           Dueling = False if agent_type == "DQN" or "DDQN" else True) for ID in npa.signal_controllers_ids] 
time_t = 0

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Deploying instance of Double Deep Q Learning Agent(s) in Junction 0
Deploying instance of Double Deep Q Learning Agent(s) in Junction 1
Deploying instance of Double Deep Q Learning Agent(s) in Junction 2
Deploying instance of Double Deep Q Learning Agent(s) in Junction 3
Deploying instance of Double Deep Q Learning Agent(s) in Junction 4
Deploying instance of Double Deep Q Learning Agent(s) in Junction 5
Deploying instance of Double Deep Q Learning Agent(s) in Junction 6
Deploying instance of Double Deep Q Learning Agent(s) in Junction 7
Deploying instance of Double Deep Q Learning Agent(s) in Junction 8
Deploying instance of Double Deep Q Learning Agent(s) in Junction 9
Deploying instance of Double Deep Q Learning Agent(s) in Junction 10
Deploying instance of Double Deep Q Learning Agent(s) in Junction 11
Deploying instance of Double Deep Q Learning Agent(s) in Junct

In [18]:
All_Vehicles = Vissim.Net.Vehicles.GetAll()

In [19]:
All_Vehicles = Vissim.Net.Vehicles.GetAll() 
for Veh in All_Vehicles:
    print(Veh.AttValue('Lane'))

15-2
21-2
1-1
12-1


In [ ]:
# Cycle through all agents and update them
Agents = SF.Agents_update(Agents, Vissim, state_type, reward_type, state_size, seconds_per_green, seconds_per_yellow, mode, 
time_t)

# Advance the game to the next second (proportionally to the simulator resolution).
for _ in range(0, timesteps_per_second):
    Vissim.Simulation.RunSingleStep()
    time_t += 1

In [10]:
signal_controllers     = toList(Vissim.Net.SignalControllers.GetAll())
signal_controllers_ids = range(0,len(signal_controllers))
signal_groups = [[] for _ in signal_controllers_ids]

In [14]:
print(signal_controllers)
print(len(signal_controllers))
print(signal_controllers_ids)
print(signal_groups)

[<COMObject GetAll>, <COMObject GetAll>, <COMObject GetAll>, <COMObject GetAll>, <COMObject GetAll>, <COMObject GetAll>, <COMObject GetAll>, <COMObject GetAll>, <COMObject GetAll>, <COMObject GetAll>, <COMObject GetAll>, <COMObject GetAll>, <COMObject GetAll>, <COMObject GetAll>]
14
range(0, 14)
[[<COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>], [<COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObject ItemByKey>, <COMObjec

In [13]:
for SC in signal_controllers_ids:
    for SG in range(1,signal_controllers[SC].SGs.Count+1):
        signal_groups[SC].append(signal_controllers[SC].SGs.ItemByKey(SG))

In [533]:
## SC01: Left Middle of the MAP. Four Way intersection. 12 signal groups. Identified by direction
##       of incoming traffic/
# 1 : East. Turn right and straight.
# 2 : North. All directions.
# 3 : West. Only turn left.
# 4 : West. Turn right and straight.
# 5 : South. All directions.
# 6 : Pedestrian located in South East.
# 7 : Pedestrian located in North East.
# 8 : Pedestrian located in North.
# 9 : Pedestrian located in North West.
# 10: Pedestrian located in South West.
# 11: Pedestrian located in South.
# 12: East. Only turn left.

model_phases_SC01 = [[1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0],\
                     [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1],\
                     [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0]]

extra_phases_SC01 = [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],\
                     [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

link_list_SC01 = [2, 40, 7, 38]
lane_list_SC01 = ['2-1', '2-2', '2-3', '40-1', '7-1', '7-2', '7-3', '38-1'] 

##########
## SC02 ##
##########
model_phases_SC02 = [[0, 1, 0, 0, 1, 0, 1, 1],\
                     [1, 0, 0, 1, 0, 0, 0, 0],\
                     [0, 0, 1, 0, 0, 1, 0, 0]]

extra_phases_SC02 = [[1, 1, 0, 0, 0, 0, 0, 0],\
                     [0, 0, 0, 1, 1, 0, 0, 0]]

link_list_SC02 = [5, 48, 70, 46]
lane_list_SC02 = ['5-1', '5-2', '5-3', '48-1', '70-1', '70-2', '70-3', '46-1']

##########
## SC03 ##
##########
model_phases_SC03 = [[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
                     [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
                     [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],\
                     [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

extra_phases_SC03 = [[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
                     [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
                     [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
                     [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

link_list_SC03 = [73, 100, 84, 95]
lane_list_SC03 = ['73-1', '73-2', '73-3', '100-1', '100-2', '100-3', '100-4',\
                  '84-1', '84-2', '84-3', '95-1', '95-2', '95-3', '95-4']

##########
## SC04 ##
##########
model_phases_SC04 = [[0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0],\
                     [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],\
                     [0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1]]

extra_phases_SC04 = [[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
                     [0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0]]

link_list_SC04 =[87, 36, 10, 34]
lane_list_SC04 = ['87-1', '87-2', '87-3', '36-1', '10-1', '10-2', '10-3', '34-1']

##########
## SC05 ##
##########
model_phases_SC05 = [[0, 1, 1, 0, 0],\
                     [1, 1, 0, 0, 0],\
                     [0, 0, 0, 1, 0]]

extra_phases_SC05 = []

link_list_SC05 = [8, 24, 13]
lane_list_SC05 = ['8-1', '8-2', '24-1', '13-1', '13-2', '13-3']

##########
## SC06 ##
##########
model_phases_SC06 = [[1, 0, 1, 0, 1, 0],\
                     [0, 1, 0, 1, 0, 1]]

extra_phases_SC06 = []

link_list_SC06 = [26, 23, 35]
lane_list_SC06 = ['26-1', '23-1', '35-1']

##########
## SC07 ##
##########
model_phases_SC07 = [[0, 1, 0, 1, 1, 1],
                     [1, 0, 1, 0, 0, 0]]

extra_phases_SC07 = []

link_list_SC07 = [51, 92, 64, 19]
lane_list_SC07 = ['51-1', '92-1', '92-2', '64-1', '19-1', '19-2']

##########
## SC08 ##
##########
model_phases_SC08 = [[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],\
                     [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],\
                     [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

extra_phases_SC08 = [[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],\
                     [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]]

link_list_SC08 =[18, 66, 16]
lane_list_SC08 = ['18-1', '18-2', '18-3', '66-1', '16-1', '16-2', '16-3']

##########
## SC09 ##
##########
model_phases_SC09 = [[1, 0, 1, 0, 0, 0, 0],\
                     [0, 1, 0, 0, 0, 0, 0]]

extra_phases_SC09 = []

link_list_SC09 = [62, 45, 44]
lane_list_SC09 = ['62-1', '45-1', '44-1']

##########
## SC10 ##
##########
model_phases_SC10 = [[0, 1, 0, 1, 1, 0, 1, 0],\
                     [1, 0, 1, 0, 0, 1, 0, 1]]

extra_phases_SC10 = []

link_list_SC10 = [60, 43, 55, 58]
lane_list_SC10 = ['60-1', '43-1', '55-1', '58-1']

##########
## SC11 ##
##########
model_phases_SC11 = [[1, 0, 1, 0, 0, 1, 0, 1],\
                     [0, 1, 0, 1, 1, 0, 1, 0]]

extra_phases_SC11 = []

link_list_SC11 = [32, 42, 30, 39]
lane_list_SC11 = ['32-1', '42-1', '30-1', '39-1']

##########
## SC12 ##
##########
model_phases_SC12 = [[1, 0, 1, 0, 0, 1, 0, 1],\
                     [0, 1, 0, 1, 1, 0, 1, 0]]

extra_phases_SC12 = []

link_list_SC12 = [29, 50, 28, 47]
lane_list_SC12 = ['29-1', '50-1', '28-1', '47-1']

##########
## SC13 ##
##########
model_phases_SC13 = [[1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1],\
                     [0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1],\
                     [0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0],\
                     [0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0]]

extra_phases_SC13 = []

link_list_SC13 = [27, 22, 25, 77]
lane_list_SC13 = ['27-1', '22-1', '22-2', '22-3', '25-1', '77-1', '77-2']

##########
## SC14 ##
##########
model_phases_SC14 = [[1, 0, 0, 1, 0, 0, 1, 0, 0],\
                     [0, 0, 1, 1, 0, 1, 0, 0, 0],\
                     [0, 1, 0, 0, 1, 1, 0, 1, 1]]

extra_phases_SC14 = []

link_list_SC14 = [68, 71, 75]
lane_list_SC14 = ['68-1', '68-2', '68-3', '71-1', '71-2', '75-1']

######################
## FINAL CONTAINERS ##
######################

## LIST WITH ALL STANDARD PHASES PER INTERSECTION
basic_phases = [model_phases_SC01, model_phases_SC02, model_phases_SC03, model_phases_SC04,\
               model_phases_SC05, model_phases_SC06, model_phases_SC07, model_phases_SC08, \
               model_phases_SC09, model_phases_SC10, model_phases_SC11, model_phases_SC12,\
               model_phases_SC13, model_phases_SC14]

## LIST WITH ALL POSSIBLE PHASES PER INTERSECTION
all_phases = [model_phases_SC01+extra_phases_SC01, model_phases_SC02+extra_phases_SC02,\
              model_phases_SC03+extra_phases_SC03, model_phases_SC04+extra_phases_SC04,\
              model_phases_SC05+extra_phases_SC05, model_phases_SC06+extra_phases_SC06,\
              model_phases_SC07+extra_phases_SC07, model_phases_SC08+extra_phases_SC08,\
              model_phases_SC09+extra_phases_SC09, model_phases_SC10+extra_phases_SC10,\
              model_phases_SC11+extra_phases_SC11, model_phases_SC12+extra_phases_SC12,\
              model_phases_SC13+extra_phases_SC13, model_phases_SC14+extra_phases_SC14]

## LIST OF NAMES OF LINKS
link_list = [link_list_SC01, link_list_SC02, link_list_SC03, link_list_SC04, link_list_SC05,\
             link_list_SC06, link_list_SC07, link_list_SC08, link_list_SC09, link_list_SC10,\
             link_list_SC11, link_list_SC12, link_list_SC13, link_list_SC14]

## LIST OF NAMES OF LANES
lane_list = [lane_list_SC01, lane_list_SC02, lane_list_SC03, lane_list_SC04, lane_list_SC05,\
             lane_list_SC06, lane_list_SC07, lane_list_SC08, lane_list_SC09, lane_list_SC10,\
             lane_list_SC11, lane_list_SC12, lane_list_SC13, lane_list_SC14]

## LIST OF LINKS AS OBJECTS
link_list_obj = [[] for _ in link_list]

for index, link_names in enumerate(link_list):
    for link in link_names:
        link_list_obj[index].append(Network.Links.ItemByKey(link))

## LIST OF LANES AS OBJECTS
lane_list_obj = [[] for _ in link_list]

for index, links_in_intersection in enumerate(link_list):
    for link in links_in_intersection:
        for lane in Network.Links.ItemByKey(link).Lanes:
            lane_list_obj[index].append(lane)

In [546]:
for i in range(300):
    Vissim.Simulation.RunSingleStep()
    
    if i % 3 == 0:
        for list_of_lanes in lane_list_obj:
            state = get_state(list_of_lanes)


In [539]:
for list_of_lanes in lane_list_obj:
    get_state(list_of_lanes)

In [534]:
def get_state(list_of_lanes):
    state = [get_queue(lane) for lane in list_of_lanes]
    return(state)

def get_queue(lane):
    vehicles_in_lane = lane.Vehs
    queue_in_lane = 0
    for vehicle in vehicles_in_lane:
        if vehicle.AttValue('InQueue') == 1:
            queue_in_lane +=1
    return(queue_in_lane)

In [537]:
get_state(lane_list_obj[0])

[0, 0, 0, 0, 0, 0, 1, 2]

In [506]:
SC_list = range(1,15)
links = [[100, 73, 84, 95]
number_lanes_SC03 = [4, 3, 3, 4]

lanes_object = [[] for _ in range(1,15)]

for SC in SC_list
    for link in links[SC]:
        for lane in range(1, number_lanes[SC][link]+1):
            lanes[SC].append(Network.Links.ItemByKey(link).Lanes.ItemByKey(lane))

lanes_in_3 = []
for i in range(1,5):
    lanes_in_3.append(Network.Links.ItemByKey(100).Lanes.ItemByKey(i))
for i in range(1,4):
    lanes_in_3.append(Network.Links.ItemByKey(73).Lanes.ItemByKey(i))
for i in range(1,4):
    lanes_in_3.append(Network.Links.ItemByKey(84).Lanes.ItemByKey(i))
for i in range(1,5):
    lanes_in_3.append(Network.Links.ItemByKey(95).Lanes.ItemByKey(i))

In [509]:
print(get_state(lanes_in_3))

[4, 2, 1, 4, 4, 2, 4, 5, 5, 0, 7, 2, 0, 0]
[4, 2, 1, 4, 4, 2, 4, 5, 5, 0, 7, 2, 0, 0]


In [371]:
signal_groups[2][19].SetAttValue("SigState", "RED")

In [ ]:
signal_controllers[13].SGs.ItemByKey(1).SetAttValue("SigState", "AMBER")
for i in range(10):
    Vissim.Simulation.RunSingleStep()

In [ ]:
for index, SC in enumerate(signal_controllers):
    print(index)
    for SG in range(1,SC.SGs.Count):
        signal_groups[index].append(SC.SGs.ItemByKey(SG+1))

In [ ]:
a = toList(signal_groups[0][0].SigHeads.GetAll())
print(a)
print(signal_controllers)

In [515]:
a = Network.Links.ItemByKey(100).Lanes
for i in a:
    print(i)

<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>


In [ ]:
## Create SignalHeadsCollection and unpack the SignalHeads into a list by SignalController
signal_heads = [[] for _ in signal_controllers_ids]
for SC in signal_controllers_ids:
    print("Signal Controller {}".format(SC))
    print("Total of {} Signal Groups".format(signal_controllers[SC].SGs.Count))
    for SG in range(signal_controllers[SC].SGs.Count):
        print("Processing Signal Group {}".format(SG))
        signal_heads[SC].append(toList(signal_groups[SC][SG].SigHeads.GetAll())[0])

In [ ]:
toList(signal_groups[2][8].SigHeads.GetAll())

In [ ]:
for SC in signal_controllers[0:15]:
    for SG in range(1, SC.SGs.Count+1):
        SC.SGs.ItemByKey(SG).SetAttValue("SigState", "GREEN")
for i in range(10):
    Vissim.Simulation.RunSingleStep()

In [ ]:
for SC in signal_controllers:
    print(SC.SGs.Count)

In [ ]:
signal_groups = [[] for _ in signal_controllers_ids]
for SC in signal_controllers_ids:
    for SG in range(1,signal_controllers[SC].SGs.Count+1):
        signal_groups[SC].append(signal_controllers[SC].SGs.ItemByKey(SG))
        
for SC in signal_controllers_ids:
    print(len(signal_groups[SC]))

In [ ]:
signal_groups[2][8]

In [ ]:
a = [[1],[2]]
b = [[3], [4]]
c = a+b
c